In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

2023-09-11 17:42:24.787511: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-11 17:42:24.788680: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 17:42:24.817364: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 17:42:24.817842: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 17:42:25.395203: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
data = pd.read_csv("ner.csv",encoding = "ISO-8859-1", nrows = 10000)
data.head()

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1,lowercase,marched,O


In [4]:
# Remove rows with NaN values
data.dropna(inplace=True)

In [5]:

# Extract features (words) and labels (tags)
X = data[["lemma", "next-lemma", "next-next-lemma", "next-next-pos", "next-next-shape", "next-next-word",
          "next-pos", "next-shape", "next-word", "pos", "prev-iob", "prev-lemma", "prev-pos", "prev-prev-iob",
          "prev-prev-lemma", "prev-prev-pos", "prev-prev-shape", "prev-prev-word", "prev-shape", "prev-word",
          "shape", "word"]]
y = data["tag"]


In [6]:
from sklearn.feature_extraction import DictVectorizer

v = DictVectorizer(sparse=False)
X = v.fit_transform(X.to_dict('records'))

classes = np.unique(y)
classes = classes.tolist()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state=0)
X_train.shape, y_train.shape

((6700, 24138), (6700,))

In [7]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=0.01)
nb.partial_fit(X_train, y_train, classes)

MultinomialNB(alpha=0.01)

In [8]:
new_classes = classes.copy()
new_classes.pop()

'O'

In [9]:
from sklearn.metrics import accuracy_score
y_pred = nb.predict(X_test)
accuracy_score(y_test, y_pred)

0.8866666666666667

In [8]:
from sklearn.metrics import classification_report
print(classification_report(y_pred=nb.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

       B-art       0.10      0.78      0.18         9
       B-eve       0.22      0.67      0.33         3
       B-geo       0.61      0.67      0.64        69
       B-gpe       0.47      0.78      0.58       102
       B-nat       0.00      0.00      0.00         0
       B-org       0.47      0.70      0.56        63
       B-per       0.61      0.68      0.64        41
       B-tim       0.69      0.88      0.77        52
       I-art       0.67      0.40      0.50        10
       I-eve       0.00      0.00      0.00         3
       I-geo       0.35      0.73      0.47        11
       I-gpe       0.10      0.33      0.15         6
       I-nat       0.00      0.00      0.00         1
       I-org       0.57      0.87      0.69        47
       I-per       0.92      1.00      0.96        66
       I-tim       0.17      0.75      0.27         4

   micro avg       0.50      0.77      0.61       487
   macro avg       0.37   

/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behav

In [9]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'alpha': [0.1, 0.5, 1.0],  # Different alpha values
    'class_prior': [None, [0.2, 0.8], [0.5, 0.5]],  # Different class_prior values
    'fit_prior': [True, False],  # Different fit_prior values
    'force_alpha': [False, True]  # Different force_alpha values
}


In [10]:
# Initialize MultinomialNB
nb = MultinomialNB()

# Initialize GridSearchCV
grid_search = GridSearchCV(nb, param_grid, cv=5, scoring='accuracy', verbose=1)

# Fit the GridSearchCV to your data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
120 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fi

Best Parameters: {'alpha': 0.5, 'class_prior': None, 'fit_prior': False, 'force_alpha': False}
Best Score: 0.9288059701492537


In [11]:
from sklearn.metrics import accuracy_score
y_pred = grid_search.predict(X_test)
accuracy_score(y_test, y_pred)

0.9312121212121212

In [12]:
print(classification_report(y_pred=grid_search.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00         9
       B-eve       0.00      0.00      0.00         3
       B-geo       0.57      0.74      0.65        69
       B-gpe       0.52      0.73      0.60       102
       B-nat       0.00      0.00      0.00         0
       B-org       0.74      0.40      0.52        63
       B-per       0.76      0.68      0.72        41
       B-tim       0.95      0.37      0.53        52
       I-art       0.00      0.00      0.00        10
       I-eve       0.00      0.00      0.00         3
       I-geo       0.00      0.00      0.00        11
       I-gpe       0.00      0.00      0.00         6
       I-nat       0.00      0.00      0.00         1
       I-org       0.75      0.70      0.73        47
       I-per       0.85      1.00      0.92        66
       I-tim       0.00      0.00      0.00         4

   micro avg       0.67      0.61      0.64       487
   macro avg       0.32   

/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behav

In [10]:
from sklearn.linear_model import PassiveAggressiveClassifier
pa =PassiveAggressiveClassifier()
pa.partial_fit(X_train, y_train, classes)

PassiveAggressiveClassifier()

In [11]:
from sklearn.metrics import accuracy_score
y_pred = pa.predict(X_test)
accuracy_score(y_test, y_pred)

0.9578787878787879

In [14]:
print(classification_report(y_pred=pa.predict(X_test), y_true=y_test, labels=new_classes))

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00         9
       B-eve       0.00      0.00      0.00         3
       B-geo       0.66      0.75      0.70        69
       B-gpe       0.87      0.70      0.77       102
       B-nat       0.00      0.00      0.00         0
       B-org       0.59      0.70      0.64        63
       B-per       0.69      0.93      0.79        41
       B-tim       1.00      0.48      0.65        52
       I-art       1.00      0.20      0.33        10
       I-eve       1.00      0.33      0.50         3
       I-geo       0.80      0.73      0.76        11
       I-gpe       1.00      0.17      0.29         6
       I-nat       0.00      0.00      0.00         1
       I-org       0.75      0.89      0.82        47
       I-per       0.96      0.98      0.97        66
       I-tim       1.00      0.50      0.67         4

   micro avg       0.77      0.72      0.74       487
   macro avg       0.64   

/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behav

In [15]:
param_grid = {
    'C': [0.1, 1.0, 10.0],  # Different values for the regularization parameter C
    'loss': ['hinge', 'squared_hinge']  # Different loss functions
}


In [16]:
# Initialize PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(pac, param_grid, cv=5, scoring='accuracy', verbose=1)


In [17]:
# Fit the GridSearchCV to your data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
print(classification_report(y_pred=pac.predict(X_test), y_true=y_test, labels=new_classes))

In [12]:
from sklearn.ensemble import RandomForestClassifier
# Create the Random Forest classifier
rf_classifier = RandomForestClassifier(random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [13]:
from sklearn.metrics import accuracy_score
y_pred = rf_classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.9518181818181818

In [20]:
classification_rep = classification_report(y_test, y_pred, labels=new_classes)
print("Classification Report:")
print(classification_rep)


Classification Report:
              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00         9
       B-eve       0.00      0.00      0.00         3
       B-geo       0.57      0.70      0.63        69
       B-gpe       0.76      0.70      0.73       102
       B-nat       0.00      0.00      0.00         0
       B-org       0.74      0.59      0.65        63
       B-per       0.74      0.83      0.78        41
       B-tim       1.00      0.62      0.76        52
       I-art       0.00      0.00      0.00        10
       I-eve       0.00      0.00      0.00         3
       I-geo       1.00      0.64      0.78        11
       I-gpe       1.00      0.33      0.50         6
       I-nat       0.00      0.00      0.00         1
       I-org       0.90      0.79      0.84        47
       I-per       0.86      0.98      0.92        66
       I-tim       0.00      0.00      0.00         4

   micro avg       0.77      0.68      0.73       487
   

/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behav

In [21]:
# Define the grid of hyperparameters to search
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 10, 20],  # Maximum depth of the trees
    
}


In [22]:
# Initialize RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(rf_classifier, param_grid, cv=3, scoring='accuracy', verbose=1)

In [23]:
# Fit the GridSearchCV to your data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


/home/prathiba/pra/VIT Chennai/paper/code/.environment/lib/python3.11/site-packages/sklearn/model_selection/_split.py:725: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
y_pred = rf_classifier.predict(X_test)

In [ ]:
classification_rep = classification_report(y_test, y_pred, labels=new_classes)
print("Classification Report:")
print(classification_rep)
